In [30]:
## make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
from typing import NamedTuple

from pa_lib.file import (
    project_dir,
    load_bin,
    load_pickle,
    load_xlsx,
    store_bin,
)
from pa_lib.data import (
    as_dtype,
    dtFactor,
    lookup,
    desc_col,
    chi2_expected,
    clean_up_categoricals,
)
from pa_lib.util import cap_words, collect, value, normalize_rows, as_percent, flatten

# display long columns completely, show more rows
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data sets

In [5]:
with project_dir("axinova"):
    ax_data = load_bin("ax_data.feather")
    spr_data = load_pickle("spr_data.pkl")
    time_codes = load_pickle("time_code_ratios.pkl")
    station_codes = load_pickle("station_code_ratios.pkl")
    global_codes = load_pickle("global_code_ratios.pkl")
    population_codes = load_pickle("population_ratios.pkl")

10:34:29 [INFO] Started loading binary file ...
10:34:29 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_data.feather
10:34:29 [INFO] ... finished loading binary file in 0.28s (0.97s CPU)
10:34:29 [INFO] Started loading pickle file ...
10:34:29 [INFO] Reading from file C:\Users\kpf\data\axinova\spr_data.pkl
10:34:29 [INFO] ... finished loading pickle file in 0.01s (0.02s CPU)
10:34:29 [INFO] Started loading pickle file ...
10:34:29 [INFO] Reading from file C:\Users\kpf\data\axinova\time_code_ratios.pkl
10:34:29 [INFO] ... finished loading pickle file in 0.01s (0.0s CPU)
10:34:29 [INFO] Started loading pickle file ...
10:34:29 [INFO] Reading from file C:\Users\kpf\data\axinova\station_code_ratios.pkl
10:34:29 [INFO] ... finished loading pickle file in 0.0s (0.02s CPU)
10:34:29 [INFO] Started loading pickle file ...
10:34:29 [INFO] Reading from file C:\Users\kpf\data\axinova\global_code_ratios.pkl
10:34:29 [INFO] ... finished loading pickle file in 0.0s (0.0s CPU)
10:34:29 [INFO] S

## Select Axinova data by any column(s)

In [122]:
def _check_selection(data, selection, allowed_columns):
    allowed_values = {}
    for column in allowed_columns:
        allowed_values[column] = data[column].cat.categories
    if set(selection.keys()) - set(allowed_columns) != set():
        raise NameError(f"Unknown column name in selection: {selection.keys()}")
    clean_selection = {}
    for column in allowed_columns:
        if column in selection:
            col_values = list(flatten(selection[column]))
            if set(col_values) - set(allowed_values[column]) != set():
                raise ValueError(
                    f"Illegal value(s) in parameter {column}: {col_values}"
                )
            clean_selection[column] = col_values
        else:
            clean_selection[column] = None
    return clean_selection


def select_data(all_data, **selection):
    select_columns = (
        "DayOfWeek Station Variable Month TimeSlot Hour "
        + "Time TimeSlot_cat StationSprache".split()
    )
    selection = _check_selection(all_data, selection, allowed_columns=select_columns)
    row_mask = pd.Series([True] * all_data.shape[0])
    for col in select_columns:
        if selection[col] is not None:
            row_mask &= all_data[col].isin(selection[col])
    return all_data.loc[row_mask].pipe(clean_up_categoricals).reset_index(drop=True)

# Look up code ratios from Axinova data

In [101]:
RatioTable = pd.DataFrame
Ratios = NamedTuple("Ratios", (("actual", RatioTable), ("expected", RatioTable), ("sd", RatioTable)))

In [102]:
def ax_population_ratios(variable: str, percent: bool = False) -> RatioTable:
    ratios = population_codes.query("Variable == @variable").pivot_table(
        values="Pop_Ratio", index="Variable", columns="Code"
    )
    return as_percent(ratios) if percent else ratios


def ax_global_ratios(variable: str, percent: bool = False) -> RatioTable:
    ratios = global_codes.query("Variable == @variable").pivot_table(
        values="Ratio", index="Variable", columns="Code"
    )
    return as_percent(ratios) if percent else ratios


def ax_station_ratios(variable: str, percent: bool = False) -> Ratios:
    actual_ratios = station_codes.query("Variable == @variable").pivot_table(
        values="Ratio", index="Station", columns="Code", fill_value=0
    )
    expected_ratios = ax_global_ratios(variable)
    if percent:
        result = Ratios(
            actual=as_percent(actual_ratios), expected=as_percent(expected_ratios)
        )
    else:
        result = Ratios(actual=actual_ratios, expected=expected_ratios)
    return result

In [103]:
def ax_ratios(
    variable: str,
    stations: str,
    reference: str = "all_stations",
    time_scale: str = "Hour",
    percent: bool = False,
) -> Ratios:
    subset = ax_data.loc[
        ax_data.Station.isin(flatten(stations)) & (ax_data.Variable == variable)
    ]
    full_index = [
        (weekday, time)
        for weekday in ax_data["DayOfWeek"].cat.categories
        for time in ax_data[time_scale].cat.categories
    ]
    actual_counts = subset.pivot_table(
        values="Value",
        index=["DayOfWeek", time_scale],
        columns="Code",
        fill_value=0,
        aggfunc="sum",
    )
    actual_counts_sd_ratios = (np.sqrt(actual_counts) / actual_counts).fillna(0)
    actual_ratios = normalize_rows(actual_counts).reindex(full_index, fill_value=0)

    if reference == "all_stations":
        expected_ratios = (
            time_codes[time_scale]
            .query("Variable == @variable")
            .pivot_table(
                values="Ratio",
                index=["DayOfWeek", time_scale],
                columns="Code",
                fill_value=0,
            )
        )
    elif reference == "station":
        expected_counts = chi2_expected(actual_counts)
        expected_ratios = normalize_rows(expected_counts).reindex(
            full_index, fill_value=0
        )
    else:
        raise ValueError(
            "Parameter 'reference' must be one of "
            + f"('station', 'all_stations'), was '{reference}'"
        )

    if percent:
        result = Ratios(
            actual=as_percent(actual_ratios),
            expected=as_percent(expected_ratios),
            sd=as_percent(actual_counts_sd_ratios),
        )
    else:
        result = Ratios(
            actual=actual_ratios, expected=expected_ratios, sd=actual_counts_sd_ratios
        )
    return result

# Look up SPR+ data split by variable

In [183]:
def spr_split(
    stations,
    variable,
    reference="station",
    time_scale="Hour",
    type="abs",
    incl_totals=True,
    decimals=1,
):
    if type not in ["abs", "rel"]:
        raise ValueError(
            f"Parameter 'type' must be one of ('abs', 'rel'), was '{type}'"
        )
    if reference not in ["station", "all_stations"]:
        raise ValueError(
            "Parameter 'reference' must be one of "
            + f"('station', 'all_stations'), was '{reference}'"
        )

    spr_counts = (
        spr_data.loc[spr_data.Station.isin(flatten(stations))]
        .groupby(["DayOfWeek", time_scale])[["Total"]]
        .agg("sum")
    )
    spr_count_sd_ratios = (np.sqrt(spr_counts) / spr_counts).fillna(0)

    ratios = ax_ratios(
        stations=stations,
        variable=variable,
        reference=reference,
        time_scale=time_scale,
        percent=False,
    )
    if type == "abs":
        code_ratios = ratios.actual
    elif type == "rel":
        code_ratios = ratios.actual - ratios.expected
    split_counts = code_ratios.mul(spr_counts.Total, axis="index").round(decimals)

    sd_ratios = np.sqrt(
        (ratios.sd ** 2).add((spr_count_sd_ratios ** 2).values, axis="index")
    )

    if incl_totals:
        split_counts.set_axis(split_counts.columns.to_list(), axis="columns", inplace=True)
        split_counts["Total"] = spr_counts.round(decimals)
        sd_ratios.set_axis(sd_ratios.columns.to_list(), axis="columns", inplace=True)
        sd_ratios["Total"] = spr_count_sd_ratios.values
    if decimals == 0:
        split_counts = split_counts.astype("int")

    return split_counts, sd_ratios

# Tests

## Station ratios

In [10]:
with value(ax_station_ratios(variable="g_220", percent=True)) as autobesitz:
    display(autobesitz.actual)
    display(autobesitz.expected)
    display(autobesitz.actual.sub(autobesitz.expected.values, axis="columns"))

Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,43,30,27
Basel SBB,44,20,36
Bellinzona,52,38,10
Bern,43,28,28
Biel/Bienne,50,25,25
Brig,51,40,9
Chur,48,35,17
Fribourg,31,56,14
Genève Aéroport,41,49,10


Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,45,29,26


Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,-2,1,1
Basel SBB,-1,-9,10
Bellinzona,7,9,-16
Bern,-2,-1,2
Biel/Bienne,5,-4,-1
Brig,6,11,-17
Chur,3,6,-9
Fribourg,-14,27,-12
Genève Aéroport,-4,20,-16


## Time ratios for one station

In [11]:
with value(ax_ratios(stations="Lausanne", variable="g_220")) as lausanne_auto:
    display(lausanne_auto.actual)
    display(lausanne_auto.expected)
    display(lausanne_auto.actual - lausanne_auto.expected)

Code              1 Auto  2+ Autos    Keines
DayOfWeek Hour                              
Monday    00    0.333711  0.384791  0.281498
          01    1.000000  0.000000  0.000000
          02    0.000000  0.000000  0.000000
          03    0.409561  0.000000  0.590439
          04    0.380052  0.000000  0.619948
...                  ...       ...       ...
Sunday    19    0.296197  0.419960  0.283843
          20    0.361548  0.423531  0.214922
          21    0.417628  0.315654  0.266718
          22    0.279258  0.384970  0.335771
          23    0.296587  0.364008  0.339405

[168 rows x 3 columns]

Code              1 Auto  2+ Autos    Keines
DayOfWeek Hour                              
Monday    00    0.379786  0.316787  0.303428
          01    0.505276  0.251623  0.243101
          02    0.374364  0.060254  0.565383
          03    0.486027  0.395825  0.118148
          04    0.285555  0.409515  0.304931
...                  ...       ...       ...
Sunday    19    0.384024  0.266989  0.348987
          20    0.379769  0.332839  0.287393
          21    0.369698  0.340051  0.290251
          22    0.459069  0.309407  0.231524
          23    0.407319  0.252926  0.339755

[168 rows x 3 columns]

Code              1 Auto  2+ Autos    Keines
DayOfWeek Hour                              
Monday    00   -0.046075  0.068004 -0.021930
          01    0.494724 -0.251623 -0.243101
          02   -0.374364 -0.060254 -0.565383
          03   -0.076466 -0.395825  0.472291
          04    0.094497 -0.409515  0.315017
...                  ...       ...       ...
Sunday    19   -0.087828  0.152971 -0.065144
          20   -0.018221  0.090692 -0.072471
          21    0.047930 -0.024398 -0.023533
          22   -0.179810  0.075563  0.104247
          23   -0.110731  0.111082 -0.000350

[168 rows x 3 columns]

## SPR+ data for one station split by ratios

In [12]:
with value(("Bern", "g_220", "Hour")) as (stat, var, scale):
    display(spr_split(stations=stat, variable=var, type="abs", time_scale=scale))
    display(spr_split(stations=stat, variable=var, type="rel", time_scale=scale))

Code            1 Auto  2+ Autos  Keines
DayOfWeek Hour                          
Monday    00       904       534     606
          01         0         0       0
          02         0         0       0
          03         0         0       0
          04         0         0       0
...                ...       ...     ...
Sunday    19     17954      9953   11323
          20     11530      7955   10888
          21      8026      7698    6904
          22      6384      3777    4581
          23      3979      2481    1779

[168 rows x 3 columns]

Code            1 Auto  2+ Autos  Keines
DayOfWeek Hour                          
Monday    00        74       -77       3
          01         0         0       0
          02         0         0       0
          03         0         0       0
          04         0         0       0
...                ...       ...     ...
Sunday    19      2027     -1775    -252
          20      -801     -1126    1927
          21     -1161       933     228
          22       399      -630     231
          23       634        17    -652

[168 rows x 3 columns]

# Case study: Find best stations / time slots for owner of multiple cars

## Population ratios

In [17]:
ax_population_ratios(variable="g_220", percent=True)

Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,44,40,16


## Ratios over all stations

In [13]:
ax_global_ratios(variable="g_220", percent=True)

Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,45,29,26


Multi-car owners are **underrepresented** at digital station panels (29% vs. 40%)

## Ratios per station

In [15]:
ax_station_ratios(variable="g_220", percent=True).actual

Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,43,30,27
Basel SBB,44,20,36
Bellinzona,52,38,10
Bern,43,28,28
Biel/Bienne,50,25,25
Brig,51,40,9
Chur,48,35,17
Fribourg,31,56,14
Genève Aéroport,41,49,10


Best stations are **Fribourg** (56%), **Zürich Flughafen** (50%), and **Genève Aéroport** (49%).

This compares to 29% over all stations, and 40% in the base population.

## Time slots at one station: Zürich HB

In [80]:
prozent = ax_ratios(
    variable="g_220", stations="Zürich HB", time_scale="Hour", percent=True
).actual
personen = spr_split(
    stations="Zürich HB", variable="g_220", time_scale="Hour", type="abs", decimals=0
)
time_analysis = prozent.merge(
    personen, left_index=True, right_index=True, suffixes=(" [%]", " [Pers.]")
)

time_analysis.loc[prozent["2+ Autos"] > 40]

1 Auto [%]  2+ Autos [%]  Keines [%]  1 Auto [Pers.]  \
DayOfWeek Hour                                                         
Tuesday   00            21            57          22             473   
          04            43            57           0             256   
Wednesday 03             0           100           0               0   
Thursday  00            24            46          30             703   
          02             0           100           0               0   
          03             0           100           0               0   
Friday    03            49            51           0             516   
          04             0           100           0               0   

                2+ Autos [Pers.]  Keines [Pers.]  Total  
DayOfWeek Hour                                           
Tuesday   00                1291             498   2263  
          04                 346               0    602  
Wednesday 03                 996               0    996  
Thursday  00                1371             874   2948  
          02                 354               0    354  
          03                 290               0    290  
Friday    03                 542               0   1058  
          04                 881               0    881

In [79]:
time_analysis.sort_values("2+ Autos [Pers.]", ascending=False).head(20)

1 Auto [%]  2+ Autos [%]  Keines [%]  1 Auto [Pers.]  \
DayOfWeek Hour                                                         
Wednesday 17            49            29          22           47721   
Tuesday   17            50            28          22           49731   
Thursday  16            46            32          22           31827   
          17            53            23          24           50138   
Monday    17            57            22          21           54772   
Tuesday   16            48            30          22           33758   
Wednesday 16            44            29          27           30522   
Monday    16            47            28          25           32478   
Friday    17            52            22          26           46920   
Sunday    16            46            24          30           34549   
Friday    18            44            24          32           33147   
Tuesday   08            52            30          18           30736   
Wednesday 18            45            22          34           35651   
Monday    08            46            29          24           26844   
Sunday    17            41            22          38           31068   
Thursday  08            47            29          24           26439   
Wednesday 08            49            29          22           27738   
Tuesday   18            52            20          28           41999   
Thursday  18            48            20          31           38453   
Friday    16            49            21          29           35551   

                2+ Autos [Pers.]  Keines [Pers.]  Total  
DayOfWeek Hour                                           
Wednesday 17               27631           21283  96636  
Tuesday   17               27317           21822  98870  
Thursday  16               22138           15377  69342  
          17               21937           23174  95249  
Monday    17               21046           20270  96088  
Tuesday   16               20761           15520  70039  
Wednesday 16               19998           18834  69354  
Monday    16               19793           17386  69656  
Friday    17               19356           23648  89924  
Sunday    16               18425           22822  75796  
Friday    18               17786           23776  74708  
Tuesday   08               17559           10579  58874  
Wednesday 18               17439           26811  79901  
Monday    08               17155           14174  58173  
Sunday    17               16693           28748  76509  
Thursday  08               16624           13682  56745  
Wednesday 08               16613           12227  56578  
Tuesday   18               16389           22615  81002  
Thursday  18               16338           24982  79773  
Friday    16               15466           21244  72261

## Time slots with CI at one station: Zürich HB

In [190]:
(counts, sd_ratios) = spr_split(
    stations="Zürich HB", variable="md_ek", time_scale="Hour", type="abs", decimals=0
)

counts.merge(
    sd_ratios, left_index=True, right_index=True, suffixes=(" [Pers]", " [sd]")
)

Keine Angabe [Pers]  Mehr als 12'000 CHF [Pers]  \
DayOfWeek Hour                                                    
Monday    00                      0                         848   
          01                      0                           0   
          02                      0                           0   
          03                      0                           0   
          04                      0                           0   
          05                    556                        4476   
          06                    974                       11515   
          07                   5208                       23665   
          08                   3227                       20352   
          09                   3983                        9554   
          10                   3541                        7749   
          11                   4383                        9895   
          12                   8607                       10742   
          13                   3463                        7899   
          14                   5828                       11745   
          15                   6207                       11917   
          16                   6037                       19267   
          17                  10800                       31224   
          18                   5395                       22676   
          19                   5507                       13267   
          20                   3920                        7802   
          21                   2010                        6638   
          22                   2592                        3595   
          23                    953                        2426   
Tuesday   00                      0                         629   
          01                      0                         294   
          02                      0                         156   
          03                      0                         248   
          04                      0                         316   
          05                    173                        5068   
          06                   1019                       12954   
          07                   6320                       23652   
          08                   3891                       17962   
          09                   2722                        9289   
          10                   3099                        6695   
          11                   3376                        9089   
          12                   8099                       15594   
          13                   2946                        8712   
          14                   4146                       12086   
          15                   3696                       13538   
          16                   6494                       21346   
          17                   7862                       34318   
          18                  10818                       19239   
          19                   5334                       13364   
          20                   3894                       10034   
          21                   3638                        7239   
          22                   2033                        3697   
          23                   1179                        1414   
Wednesday 00                    112                         951   
          01                      0                         257   
          02                      0                           0   
          03                      0                         996   
          04                      0                         603   
          05                      0                        3183   
          06                   1693                       11941   
          07                   7229                       22712   
          08                   6300                       16776   
          09                   3776                        9497